### LeNet
* 专为灰度图设计
* 输入 (1x32x32)
*      ↓
* Conv1 (6x28x28) + ReLU
*      ↓
* MaxPool1 (6x14x14)
*      ↓
* Conv2 (16x10x10) + ReLU
*      ↓
* MaxPool2 (16x5x5)
*      ↓
* Flatten (400)
*      ↓
* FC1 (120) + ReLU
*      ↓
* FC2 (84) + ReLU
*      ↓
* FC3/Output (10)

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Pad(2),  # 28x28 -> 32x32，用0填充
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST(root='./data', train=True, 
                               transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, 
                              transform=transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 43.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.12MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.84MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.2MB/s]


In [6]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [7]:
net = torch.nn.Sequential(
    nn.Conv2d(1,6,5,1,0),    #(1,32,32) -> (6,28,28)
    nn.ReLU(),
    nn.MaxPool2d(2),         #(6,28,28) -> (6,14,14)
    #如果只给一个参数，PyTorch会把它当作kernel_size，
    #并且自动设置stride=kernel_size

    nn.Conv2d(6,16,5,1,0),   #(6,14,14) -> (16,10,10)
    nn.ReLU(),
    nn.MaxPool2d(2),         #(16,10,10) -> (16,5,5)

    nn.Flatten(),            # 400
    nn.Linear(400,120),
    nn.ReLU(),

    nn.Linear(120,84),
    nn.ReLU(),

    nn.Linear(84,10)
)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [9]:
X = torch.rand(size=(1,1,32,32),dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'\t output: \t',X.shape)

Conv2d 	 output: 	 torch.Size([1, 6, 28, 28])
ReLU 	 output: 	 torch.Size([1, 6, 28, 28])
MaxPool2d 	 output: 	 torch.Size([1, 6, 14, 14])
Conv2d 	 output: 	 torch.Size([1, 16, 10, 10])
ReLU 	 output: 	 torch.Size([1, 16, 10, 10])
MaxPool2d 	 output: 	 torch.Size([1, 16, 5, 5])
Flatten 	 output: 	 torch.Size([1, 400])
Linear 	 output: 	 torch.Size([1, 120])
ReLU 	 output: 	 torch.Size([1, 120])
Linear 	 output: 	 torch.Size([1, 84])
ReLU 	 output: 	 torch.Size([1, 84])
Linear 	 output: 	 torch.Size([1, 10])


In [14]:
def fit(epochs,net,train_loader,test_loader,opt,criterion):
    for epoch in range(epochs):
        net.train()
        for x,y in train_loader:
            pred = net(x)
            loss = criterion(pred,y)
            loss.backward()
            opt.step()
            opt.zero_grad()
        net.eval()    
        with torch.no_grad():
            t_loss = []
            right = []
            num = []
            for x,y in test_loader:
                pred = net(x)
                _, predicted = torch.max(pred, 1)
                correct = (predicted == y).sum().item()
                right.append(correct)
                num.append(x.size(0))
                loss = criterion(pred,y)
                t_loss.append(loss.item())
        print("epoch:",epoch+1,"\t loss:",sum(t_loss)/len(t_loss))
        print("right:",sum(right),"/",sum(num))
            
    
fit(10,net,train_loader,test_loader,optimizer,criterion)

epoch: 1 	 loss: 0.040376017932604265
right: 9906 / 10000
epoch: 2 	 loss: 0.0309088717476093
right: 9919 / 10000
epoch: 3 	 loss: 0.06722456477965934
right: 9840 / 10000
epoch: 4 	 loss: 0.03485879670701209
right: 9908 / 10000
epoch: 5 	 loss: 0.038442252503548796
right: 9901 / 10000
epoch: 6 	 loss: 0.046332413987341114
right: 9886 / 10000
epoch: 7 	 loss: 0.04794373469113292
right: 9879 / 10000
epoch: 8 	 loss: 0.05014491786407893
right: 9891 / 10000
epoch: 9 	 loss: 0.04700231079988361
right: 9889 / 10000
epoch: 10 	 loss: 0.05483658829930282
right: 9891 / 10000
